<a href="https://colab.research.google.com/github/karsarobert/NLP_2024/blob/main/04/NLP2024_04_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Természetes nyelvfeldolgozás
# PTE Gépi tanulás III.

## 1. Előadás: BERT, finomhangolás
### 2024. február 28.


In [ ]:
!pip install datasets evaluate transformers[sentencepiece] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.5 MB/s eta 0:00:00


# Finomhangoljuk az előképzett modellt

Az előre betanított modellek használatának jelentős előnyei vannak. Csökkenti a számítási költségeket, a szénlábnyomot, és lehetővé teszi, hogy a legmodernebb modelleket anélkül használhassa, hogy azokat a nulláról kellene betanítania. A 🤗 Transformers több ezer előre betanított modellhez biztosít hozzáférést számos feladathoz. Amikor egy előre betanított modellt használ, azt egy, a saját feladatára jellemző adathalmazon tovább tanítja. Ezt a folyamatot finomhangolásnak (fine-tuning) hívják, ami egy hihetetlenül hatékony tanítási technika. Ebben az útmutatóban egy előre betanított modellt fog finomhangolni a választott mélytanulási keretrendszerrel:

* Előre betanított modell finomhangolása a 🤗 Transformers [Trainer] segítségével ([https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)).
* Előre betanított modell finomhangolása TensorFlow-ban Keras-szal.
* Előre betanított modell finomhangolása natív PyTorch segítségével.


* **Előretaníott modell (pretrained model):** Egy olyan gépi tanulási modell, ami már egy általános adathalmazon keresztülment a kezdeti betanításon, így rendelkezik alapvető tudással és képességekkel.
* **Finomhangolás (fine-tuning):** Egy előre betanított modell további tanítása, specifikusabb feladatokra szabva.


Before you can fine-tune a pretrained model, download a dataset and prepare it for training. The previous tutorial showed you how to process data for training, and now you get an opportunity to put those skills to the test!

Begin by loading the [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) dataset:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))


dataset["train"][100]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Mint már tudod, szükség van egy tokenizálóra, amely feldolgozza a szöveget, és tartalmaz egy kitöltési és csonkítási stratégiát a változó szekvenciahosszúságok kezelésére. Az adathalmaz egy lépésben történő feldolgozásához használd a 🤗 Datasets map módszert, hogy egy előfeldolgozó függvényt alkalmazz a teljes adathalmazra:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Ha szeretné, létrehozhatja a teljes adathalmaz egy kisebb részhalmazát a finomhangoláshoz, hogy csökkentse a szükséges időt:

## Képzés

🤗 A Transformers biztosít egy [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) osztályt, amely a 🤗 Transformers modellek képzésére optimalizált, megkönnyítve a képzés megkezdését a saját képzési ciklus kézi írása nélkül. A [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) API a képzési opciók és funkciók széles skáláját támogatja, mint például a naplózás, a gradiens felhalmozás és a vegyes pontosság.

Kezdje a modell betöltésével, és adja meg a várható címkék számát. A Yelp Review [adatkészlet-kártya](https://huggingface.co/datasets/yelp_review_full#data-fields) alapján tudja, hogy öt címke van:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<Tip>

You will see a warning about some of the pretrained weights not being used and some weights being randomly
initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

</Tip>

### Képzési hiperparaméterek

Ezután hozzon létre egy [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) osztályt, amely tartalmazza az összes beállítható hiperparamétert, valamint a különböző képzési lehetőségek aktiválásához szükséges jelzőket. Ehhez a bemutatóhoz kezdheti az alapértelmezett képzési [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) beállításokkal, de nyugodtan kísérletezzen ezekkel, hogy megtalálja az optimális beállításokat.

Adja meg, hogy hová mentse a képzés ellenőrzőpontjait:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### A modell értékelése

A [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) nem értékeli automatikusan a modell teljesítményét a képzés során. Át kell adnia a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) függvényt a metrikák kiszámításához és jelentéséhez. A [🤗 Evaluate](https://huggingface.co/docs/evaluate/index) könyvtár biztosít egy egyszerű [`accuracy`](https://huggingface.co/spaces/evaluate-metric/accuracy) függvényt, amelyet az [evaluate.load](https://huggingface.co/docs/evaluate/main/en/package_reference/loading_methods#evaluate.load) függvénnyel tölthet be (további információkért lásd ezt a [quicktour](https://huggingface.co/docs/evaluate/a_quick_tour) függvényt):

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Hívja a `compute`-t a `metric`-nél, hogy kiszámítsa a jóslatok pontosságát. Mielőtt átadnád az előrejelzéseidet a `compute`-nak, át kell alakítanod az előrejelzéseket logaritmussá (ne feledd, hogy minden 🤗 Transformers modell logaritmussal tér vissza):

In [ ]:
# Metrika számító függvény definiálása, amely kiértékeli a modell predikcióit.
def compute_metrics(eval_pred):
    # A modell predikcióinak és a címkéknek kicsomagolása a bemeneti értékből
    logits, labels = eval_pred

    # Predikciók meghatározása a legnagyobb valószínűség szerinti osztály alapján
    predictions = np.argmax(logits, axis=-1)

    # Metrika számítása a predikciók és a címkék alapján
    return metric.compute(predictions=predictions, references=labels)


Ha a finomhangolás során szeretné nyomon követni a kiértékelési metrikákat, adja meg a `evaluation_strategy` paramétert a képzési argumentumokban, hogy minden epocha végén jelentse a kiértékelési metrikát:

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Trainer

Hozzon létre egy [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) objektumot a modellel, a képzési argumentumokkal, a képzési és tesztadathalmazokkal, valamint az értékelő függvénnyel:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Ezután finomhangolja a modellt a [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) meghívásával:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.159043,0.515000
2,No log,0.989752,0.581000
3,No log,1.054761,0.594000


TrainOutput(global_step=375, training_loss=0.9578626302083333, metrics={'train_runtime': 382.3427, 'train_samples_per_second': 7.846, 'train_steps_per_second': 0.981, 'total_flos': 789354427392000.0, 'train_loss': 0.9578626302083333, 'epoch': 3.0})